In [1]:
!pip3 install git+https://github.com/Centre-automatique-et-systemes/learn_observe_KKL.git gwpy &> /dev/null
!pip3 install git+https://github.com/aliutkus/torchinterp1d.git gwpy &> /dev/null

In [3]:
import sys ; sys.path.append('../')
from learn_KKL.luenberger_observer import LuenbergerObserver
from learn_KKL.system import RevDuffing
from learn_KKL.learner import Learner

In [ ]:
# Generate the data
system = RevDuffing()
data = system.generate_mesh([-1., 1.], 50000)

# Create the observer (autoencoder design)
observer = LuenbergerObserver(dim_x=2, dim_y=1)
observer.set_dynamics(system)

# Train using the learner class
learner = Learner(observer,data)
learner.train()

====> Epoch: 1 done! LR: 0.001
====> Epoch: 2 done! LR: 0.001


In [ ]:
# Simulate one test trajectory

import torch 
tsim = (0, 60)
dt = 1e-2
tq, simulation = system.simulate(torch.tensor([[0.5],[-0.5]]), tsim, dt)
measurement = observer.h(
        simulation[:, :observer.dim_x, 0].T).T
y = torch.cat((tq.unsqueeze(1), measurement), dim=1)

# Predict from measurement
estimation = observer.predict(y, tsim, dt)

In [ ]:
import matplotlib.pyplot as plt
# Plot ground truth and estimation
for i in range(simulation.shape[1]):
        plt.plot(tq, simulation[:, i, 0], label=rf'$x_{i+1}$')
        plt.plot(tq, estimation[:, i].detach().numpy(),
                 label=rf'$\hat{{x}}_{i+1}$')
        plt.legend()
        plt.show()